In [0]:
import pandas as pd
import numpy as np
import altair as alt
import requests
from bs4 import BeautifulSoup
import time
import re

In [0]:
goodm = []
badm = []
medm = []

We web scrapped from 3 websites off of imdb to get the top 250 movies, bottom 250 movies, as well as 300 mediocre movies. We wanted to get the list of movies from all the same website so that the judging of the movie ranking would be consistent. 

In [0]:
for i in range(1, 4):
  response = requests.get("https://www.imdb.com/list/ls068082370/?sort=list_order,asc&st_dt=&mode=detail&page=%d"%i) 
  soup =  BeautifulSoup(response.content, "html.parser")
  goodm += soup.find_all("div", class_ = "lister-item mode-detail")

In [0]:
for i in range(1, 4):
  response = requests.get("https://www.imdb.com/list/ls006266952/?sort=list_order,asc&st_dt=&mode=detail&page=%d"%i)
  soup = BeautifulSoup(response.content, "html.parser")
  medm += soup.find_all("div", class_ = "lister-item mode-detail")

In [0]:
for i in range(1, 251, 50):
  response = requests.get("https://www.imdb.com/search/title/?groups=bottom_250&sort=user_rating,asc&start=%d&ref_=adv_nxt"%i)
  soup = BeautifulSoup(response.content, "html.parser")
  badm += soup.find_all("div", class_ = "lister-item-content")

Our dataframe will consist of the features:

types: which group the movie was webscrapped from: good, mediocre, bad

title: the title of the movie

year: the year the movie was made

rating: the rating given to the movie: PG-13, R, PG, etc

runtime: the runtime of the movie

genre: the genre of the movie: action, comedy, adventure, etc

star-rating: the rating given by the user

metascore: the rating given by the critic

description: the description of the movie

director: the director of the movie

cast: the list of actors and actresses in the movie

votes: the number of people that voted for the movie

gross: the earnings of that movie with adjustment for inflation

In [0]:
df_movies = pd.DataFrame(columns = ["types", "title", "year", "rating",
                                    "runtime", "genre", "star-rating",
                                    "metascore", "description", "director", 
                                    "cast", "votes", "gross"])
types = []
title =[]
year = []
rating = []
runtime = []
genre = []
star_rating = []
metascore = []
description = []
directors = []
cast = []
votes = []
gross = []

The three chunks below show the code for webscrapping from the website. Even though the pages were all from imdb, the pages were coded differently which was why we needed to do 3 individual for loops for each page we scrapped. We also did some data cleaning in here as well since some of the variables had extra symbols or letters next to the values. We also adjusted for inflation in the gross variable.

In [0]:
for div in goodm:
  title_tags = div.find_all("a", href = re.compile("title"))
  title.append(title_tags[1].text)

  yr = div.find("span", class_ = "lister-item-year text-muted unbold").text
  yr = yr.replace("(I) ","")
  yr = yr.replace("(","")
  yr = yr.replace(")","")
  year.append(yr)

  rating.append(("-1" if div.find("span", class_ = "certificate") is None 
                 else div.find("span", class_ = "certificate").text))
  
  rt = (div.find("span", class_ = "runtime").text)
  rt = rt.replace(" min", "")
  runtime.append(rt)

  g = (div.find("span", class_ = "genre").text)
  g = g.replace("\n","")
  g = g.replace(" ", "")
  genre.append(g)
  
  star_rating.append(div.find("span", class_ = "ipl-rating-star__rating").text)
  
  ms = (("nan" if div.find("span", class_ = "metascore") is None
                    else div.find("span", class_ = "metascore").text))
  ms = ms.replace(" ","")
  metascore.append(ms)

  d = (div.find("p", class_ = "").text)
  d = d.replace("\n","")
  description.append(d)

  spans = div.find_all("span", class_ = "")

  v = (spans[1].text)

  gr = (spans[-1].text)
  gr = gr.replace("$","")
  gr = gr.replace("M","")
  if gr == v:
    gr = "nan"
  if gr != "nan":
    gr = float(gr)
    gr = gr * (10 ** 6)
    response = requests.get("https://inflation-api.herokuapp.com/api/?value="+ str(gr) +"&year="+yr)
    resp = response.json()
    gr = resp["response"]["adjustedValue"]
  v = v.replace(",","")
  votes.append(v)
  gross.append(gr)
  

  name_tags = div.find_all("a", href = re.compile("name"))

  director = name_tags[0].text
  directors.append(director)
  members = name_tags[1:]
  for i in range(len(members)):
    members[i] = members[i].text
  members = " ".join(members)
  cast.append(members)
  types.append("good")


In [0]:
for div in medm:
  title_tags = div.find_all("a", href = re.compile("title"))
  title.append(title_tags[1].text)

  yr = div.find("span", class_ = "lister-item-year text-muted unbold").text
  yr = yr.replace("(I) ","")
  yr = yr.replace("(","")
  yr = yr.replace(")","")
  year.append(yr)

  rating.append(("-1" if div.find("span", class_ = "certificate") is None 
                 else div.find("span", class_ = "certificate").text))
  
  rt = (div.find("span", class_ = "runtime").text)
  rt = rt.replace(" min", "")
  runtime.append(rt)

  g = (div.find("span", class_ = "genre").text)
  g = g.replace("\n","")
  g = g.replace(" ", "")
  genre.append(g)
  
  star_rating.append(div.find("span", class_ = "ipl-rating-star__rating").text)
  
  ms = (("nan" if div.find("span", class_ = "metascore") is None
                    else div.find("span", class_ = "metascore").text))
  ms = ms.replace(" ","")
  metascore.append(ms)

  d = (div.find("p", class_ = "").text)
  d = d.replace("\n","")
  description.append(d)

  spans = div.find_all("span", class_ = "")

  v = (spans[1].text)

  gr = (spans[-1].text)
  gr = gr.replace("$","")
  gr = gr.replace("M","")
  if gr == v:
    gr = "nan"
  if gr != "nan":
    gr = float(gr)
    gr = gr * (10 ** 6)
    response = requests.get("https://inflation-api.herokuapp.com/api/?value="+ str(gr) +"&year="+yr)
    resp = response.json()
    gr = resp["response"]["adjustedValue"]
  v = v.replace(",","")
  votes.append(v)
  gross.append(gr)
  

  name_tags = div.find_all("a", href = re.compile("name"))

  director = name_tags[0].text
  directors.append(director)
  members = name_tags[1:]
  for i in range(len(members)):
    members[i] = members[i].text
  members = " ".join(members)
  cast.append(members)
  types.append("med")


In [0]:
for div in badm:
  title_tags = div.find_all("a", href = re.compile("title"))
  title.append(title_tags[0].text)

  yr = div.find("span", class_ = "lister-item-year text-muted unbold").text
  yr = yr.replace("(I) ","")
  yr = yr.replace("(","")
  yr = yr.replace(")","")
  year.append(yr)

  rating.append(("nan" if div.find("span", class_= "certificate") is None 
                 else div.find("span", class_="certificate").text))
  
  rt = (div.find("span", class_ = "runtime").text)
  rt = rt.replace(" min", "")
  runtime.append(rt)

  g = (div.find("span", class_ = "genre").text)
  g = g.replace("\n","")
  g = g.replace(" ", "")
  genre.append(g)
  
  star_rating.append("nan" if div.find("strong") is None
                     else div.find("strong").text)
  
  ms = (("nan" if div.find("span", class_ = "metascore") is None
                    else div.find("span", class_ = "metascore").text))
  ms = ms.replace(" ","")
  metascore.append(ms)

  d = (div.find("p", class_ = "").text)
  d = d.replace("\n","")
  description.append(d)

  spans = div.select('span[name^=\"nv\"]')

  if len(spans) > 0:
    v = (spans[0].text)
    gr = "nan"
    if len(spans) > 1:
      gr = (spans[1].text)
      gr = gr.replace("$","")
      gr = gr.replace("M","")
    if gr != "nan":
      gr = float(gr)
      gr = gr * (10 ** 6)
      response = requests.get("https://inflation-api.herokuapp.com/api/?value="+ str(gr) +"&year="+yr)
      resp = response.json()
      gr = resp["response"]["adjustedValue"]
    v = v.replace(",","")
    votes.append(v)
    gross.append(gr)
  else:
    votes.append("nan")
    gross.append("nan")
  

  name_tags = div.find_all("a", href = re.compile("name"))

  director = name_tags[0].text
  directors.append(director)
  members = name_tags[1:]
  for i in range(len(members)):
    members[i] = members[i].text
  members = " ".join(members)
  cast.append(members)
  types.append("bad")

Once we webscrapped all the data for the variables, we assigned them into the dataframe. 

In [0]:
df_movies["types"] = types
df_movies["title"] = title
df_movies["year"] = year
df_movies["rating"] = rating
df_movies["runtime"] = runtime
df_movies["genre"] = genre
df_movies["star-rating"] = star_rating
df_movies["metascore"] = metascore
df_movies["description"] = description
df_movies["director"] = directors
df_movies["cast"] = cast
df_movies["votes"] = votes
df_movies["gross"] = gross

We had to change a lot of the features to a numeric type for future plots and analysis. We also noticed that the variable year had other letters and words attached, so we had to rid of those. 

In [0]:
df_movies["star-rating"] = df_movies["star-rating"].astype(float)
df_movies["metascore"] = df_movies["metascore"].astype(float)
df_movies["votes"] = df_movies["votes"].astype(int)
df_movies["gross"] = df_movies["gross"].astype(float)
df_movies["runtime"] = df_movies["runtime"].astype(int)
df_movies["types"] = df_movies["types"].astype('category')

In [0]:
df_movies["year"] = df_movies["year"].str.replace("I","")
df_movies["year"] = df_movies["year"].str.replace(" ","")
df_movies["year"] = df_movies["year"].str.replace("TVMovie","")

In [0]:
df_movies

,types,title,year,rating,runtime,genre,star-rating,metascore,description,director,cast,votes,gross
0,good,The Shawshank Redemption,1994,R,142,Drama,9.3,80.0,Two imprisoned men bond over a number of y...,Frank Darabont,Tim Robbins Morgan Freeman Bob Gunton William ...,2200597,4.608596e+07
1,good,The Godfather,1972,R,175,"Crime,Drama",9.2,100.0,The aging patriarch of an organized crime ...,Francis Ford Coppola,Marlon Brando Al Pacino James Caan Diane Keaton,1516220,7.763191e+08
2,good,The Dark Knight,2008,PG-13,152,"Action,Crime,Drama",9.0,84.0,When the menace known as the Joker wreaks ...,Christopher Nolan,Christian Bale Heath Ledger Aaron Eckhart Mich...,2182182,5.987053e+08
3,good,The Godfather: Part II,1974,R,202,"Crime,Drama",9.0,90.0,The early life and career of Vito Corleone...,Francis Ford Coppola,Al Pacino Robert De Niro Robert Duvall Diane K...,1062183,2.801075e+08
4,good,Pulp Fiction,1994,R,154,"Crime,Drama",8.9,94.0,"The lives of two mob hitmen, a boxer, a ga...",Quentin Tarantino,John Travolta Uma Thurman Samuel L. Jackson Br...,1729312,1.755137e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,bad,The Twilight Saga: New Moon,2009,PG-13,130,"Drama,Fantasy,Romance",4.7,44.0,"PG-13|130 min|Drama, Fantasy, Romance",Chris Weitz,Kristen Stewart Robert Pattinson Taylor Lautne...,255321,3.332654e+08
796,bad,Zoolander 2,2016,PG-13,101,"Action,Adventure,Comedy",4.7,34.0,"PG-13|101 min|Action, Adventure, Comedy ...",Ben Stiller,Ben Stiller Owen Wilson Penélope Cruz Will Fer...,62148,2.885000e+07
797,bad,God's Not Dead,2014,PG,113,Drama,4.7,16.0,PG|113 min|Drama,Harold Cronk,Shane Harper Kevin Sorbo David A.R. White Dean...,38989,6.186379e+07
798,bad,Meet the Blacks,2016,R,94,"Comedy,Horror",4.8,26.0,"R|94 min|Comedy, Horror",Deon Taylor,Mike Epps Zulay Henao Alex Henderson Bresha Webb,35249,9.090000e+06
